In [1]:
import pandas as pd # 데이터 전처리
import numpy as np # 데이터 전처리

#from tqdm import tqdm_notebook
import time

import matplotlib.pyplot as plt # 데이터 시각화
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler

from datetime import datetime, timedelta # 시간 데이터 처리

Using TensorFlow backend.


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("submission_1002.csv")

In [3]:
test['Time'] = pd.to_datetime(test['Time']) 
test = test.set_index('Time')
test.head()


,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017-07-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.275,0.021,NaN
2017-07-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.222,0.021,NaN
2017-07-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.237,0.021,NaN
2017-07-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.229,0.020,NaN
2017-07-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.202,0.021,NaN


In [4]:
class CustomHistory(keras.callbacks.Callback):
    def init(self):
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, batch, logs={}):
        self.train_loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))

In [5]:
def create_dataset(signal_data, look_back=1):
    dataX, dataY = [], []
    for i in range(len(signal_data)-look_back):
        dataX.append(signal_data[i:(i+look_back), 0])
        dataY.append(signal_data[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

class CustomHistory(keras.callbacks.Callback):
    def init(self):
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, batch, logs={}):
        self.train_loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))

In [6]:
test2 = pd.read_csv('test2.csv')

In [7]:

results = pd.DataFrame( np.zeros((24,200)) , columns=test2.columns[1:])

for idx, name in enumerate(test2.columns[1:]):
    rest = test2[name].copy()
    l = list()
    for i in range(0,8760,24):
        if not pd.isnull(rest[i]):
            l = rest[i:]
            break
    l = np.asarray(l)
    l = np.reshape(l,(len(l),1))

    size = int(len(l)/24)
    #train = l
    train = l[:int(size*2/4)*24]
    val = l[int(size*2/4)*24:int(size*3/4)*24]
    test = l[int(size*3/4)*24:]

    train = np.asarray(train)
    test = np.asarray(test)
    val = np.asarray(val)

    look_back = 24

    # 데이터셋 생성
    x_train, y_train = create_dataset(train, look_back)
    x_val, y_val = create_dataset(val, look_back)
    x_test, y_test = create_dataset(test, look_back)

    print(x_train.shape, y_train.shape)

    # 데이터셋 전처리
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_val = np.reshape(x_val, (x_val.shape[0], x_val.shape[1], 1))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    print(x_train.shape)

    # 2. 모델 구성하기
    model = Sequential()
    model.add(LSTM(32, batch_input_shape=(1, look_back, 1), stateful=True))
    model.add(Dropout(0.3))
    model.add(Dense(1))

    # 3. 모델 학습과정 설정하기
    model.compile(loss='mean_squared_error', optimizer='adam')

    # 4. 모델 학습시키기
    custom_hist = CustomHistory()
    custom_hist.init()
    early_stopping = EarlyStopping(monitor = 'val_loss', patience=1, verbose=1 )

    for i in range(100):
        model.fit(x_train, y_train, epochs=1, batch_size=1, shuffle=False, callbacks=[early_stopping, custom_hist], validation_data=(x_val, y_val))

#     # 5. 학습과정 살펴보기
#     plt.plot(custom_hist.train_loss)
#     plt.plot(custom_hist.val_loss)
#     plt.ylim(0.0, 0.15)
#     plt.ylabel('loss')
#     plt.xlabel('epoch')
#     plt.legend(['train', 'val'], loc='upper left')
#     plt.show()

#     # 6. 모델 평가하기
#     trainScore = model.evaluate(x_train, y_train, batch_size=1, verbose=0)
#     model.reset_states()
#     print('Train Score: ', trainScore)
#     valScore = model.evaluate(x_val, y_val, batch_size=1, verbose=0)
#     model.reset_states()
#     print('Validataion Score: ', valScore)
#     testScore = model.evaluate(x_test, y_test, batch_size=1, verbose=0)
#     model.reset_states()
#     print('Test Score: ', testScore)

    # 7. 모델 사용하기
    look_ahead = 24
    xhat = x_test[0]
    predictions = np.zeros((look_ahead,1))
    
    for i in range(look_ahead):
        prediction = model.predict(np.array([xhat]), batch_size=1)
        predictions[i] = prediction
        xhat = np.vstack([xhat[1:],prediction])



    plt.figure(figsize=(12,5))
    plt.plot(np.arange(look_ahead),predictions,'r',label="prediction")
    plt.plot(np.arange(look_ahead),y_test[:look_ahead],label="test function")
    plt.legend()
    plt.show()
    
    results[name] = predictions
#     print(predictions)
#     print(results[name])
#     print(results)
    
    
results.to_csv("./result.csv")


(840, 24) (840,)
(840, 24, 1)
Train on 840 samples, validate on 408 samples
Epoch 1/1
158/840 [====>.........................] - ETA: 7s - loss: 0.6075

KeyboardInterrupt: 